## Description
  #### Class V single hull dry bulk vessel: 110x11.4x3.5m,DWT=3000ton, Pinstall=1750kw
  ####  (cargo amount 75000ton)
  #### up-downstream,
  #### engine age
  #### high-low water level
  #### actual draft&loading weight


## Import packages

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math 

## Input Variables: velocity, water depth,draft, engine age, distance

In [2]:
#Variables
#V_0 = np.linspace(0.1, 8, 200)
V_0 = 4                      #m/s   downstream=11.376km/h,3.16M/S    upstream=15km/h,4.16m/s  NO CURRENT 13.176km/h 3.66m/s
V_0_km = V_0 * 3.6              #km/h

h = 6  # Kelheim-Passau high 5.4m, low 3.5m; Passau-Linz high 5.2m, low 4.0m
T = 1.4                # actrual draft of different load weight

c_year = 2020   # new engine 2020, old engine 1990
distance= 179 # km (Kelheim-Passau 179km,Passau-Linz 104km)

## Fixed input values

In [3]:
rho = 1000       # kg/m³
g = 9.81        # m/s^2

#ship related 
L = 110                 # values of M8
B = 11.4                # values of M8


c_stern = 0         
C_B = 0.85              #block coefficient 
viscosity = 1*10**(-6)  #kinematic viscosity
x = 2                   #number of screws

#power efficienty

P_installed = 1750 #kW 
eta_0 = 0.6      #open water eff
eta_r = 0.98     #relative rotative eff
eta_t = 0.98     #transmission eff
eta_g = 0.96     #gearing eff
L_w=3

In [4]:
#Ship related values, calculated from fixed input values

C_M = 1.006 - 0.0056 * C_B **(-3.56)       #midshipcoefficient
C_wp = (1+2 * C_B)/3                       #waterplane coefficient
C_p = C_B / C_M                            #prismatic coefficient

delta = C_B * L * B * T                    #water displacement

lcb = -13.5 + 19.4 * C_p                   #longitudinal position of the centre of buoyancy as % of L
L_R = L * (1 - C_p + (0.06 * C_p * lcb)/(4 * C_p -1))   #Length of the run

A_T = 0.2 * B * T     #area immersed transom

#Total wet surface area (S_T), wet area of appendages (S_APP) and wet area of flat bottom (S_B)
S_T = L*(2*T + B) * np.sqrt(C_M) * (0.453 + 0.4425 * C_B - 0.2862* C_M - 0.003467 * (B/T) + 0.3696 * C_wp) #+ 2.38 * (A_BT / C_B)
S_APP = 0.05 * S_T
S_B = L * B

#Screw diameter
D_s = 0.7 * T

#Viscous resistance (1+k1)
c_14 = 1 + 0.0011 * c_stern
one_k1 = 0.93+0.487 * c_14 * ((B/L)**1.068) * ((T/L)**0.461) * ((L/L_R)**0.122) * (((L**3)/delta)**0.365) * ((1 - C_p)**(-0.604))

#appendage resistance factor(1+k2)
one_k2 = 2.5 


In [5]:
one_k1

1.1962200696113838

#  Resistance estimation

## (1)Frictional resistance: Rf (1 + k1) , Rapp

In [6]:
#### Method of Zeng(2019) incorporating the shallow water effect in the frictional resistance term #######

#def frictional_resistance(h):
R_e = V_0 * L / viscosity
D = h - T #distance from bottom ship to the bottom of the fairway

Cf_0 = 0.075 / ((np.log10(R_e)-2)**2)
    
Cf_proposed = (0.08169/((np.log10(R_e) - 1.717)**2)) * (1 + (0.003998/(np.log10(R_e) - 4.393)) * (D/L)**(-1.083))

a = 0.042612 * np.log10(R_e) + 0.56725
    
Cf_katsui = 0.0066577 / ((np.log10(R_e) - 4.3762)**a)

if h/T <= 4:
    V_B = 0.4277 * V_0 * np.exp((h / T)**(-0.07625))      # bottom accelated velocity because of shallow water effect
else:
    V_B = V_0

C_f = Cf_0 + (Cf_proposed - Cf_katsui) * (S_B / S_T) * (V_B / V_0)**2

R_f = (C_f * 0.5 * rho * (V_0**2) * S_T) / 1000 #kN
    
R_APP = (0.5 * rho * (V_0**2) * S_APP * one_k2 * C_f) / 1000 #kN
    
Rf_onek1 = R_f*one_k1
    
print(Rf_onek1,R_APP, R_f,) 

21.983263679219423 2.2971592181989893 18.37727374559192


## (2) Wave resistance: Rw

In [7]:
############ Additional wave making resistance - Karpov a**  V2 #############


#def karpov2(h, F_n):
    
#The Froude number used in the Karpov method is the depth related froude number F_nh
    
 #The different alpha** curves are determined with a sixth power polynomial approximation in Excel
 #A distinction is made between different ranges of Froude numbers, because this resulted in a better approximation of the curve
F_n = V_0 / np.sqrt(g * h)
   
if F_n <= 0.4:
        
    if 0 <= h/T < 1.75:
        alpha_xx = (-4*10**(-12)) * F_n**3 - 0.2143 * F_n**2 -0.0643 * F_n + 0.9997
    elif 1.75 <= h/T < 2.25:
        alpha_xx = -0.8333 * F_n**3 + 0.25 * F_n**2 - 0.0167 * F_n + 1
    elif 2.25 <= h/T < 2.75:
        alpha_xx = -1.25 * F_n**4 + 0.5833 * F_n**3 - 0.0375 * F_n**2 - 0.0108 * F_n + 1
    elif h/T >= 2.75:
        alpha_xx = 1
    
elif F_n > 0.4:
    if 0 <= h/T < 1.75:
        alpha_xx = -0.9274 * F_n**6 + 9.5953 * F_n**5 - 37.197 * F_n**4 + 69.666 * F_n**3 - 65.391 * F_n**2 + 28.025 * F_n - 3.4143
    elif 1.75 <= h/T < 2.25:
        alpha_xx = 2.2152 * F_n**6 - 11.852 * F_n**5 + 21.499 * F_n**4 - 12.174 * F_n**3 - 4.7873 * F_n**2 + 5.8662 * F_n - 0.2652
    elif 2.25 <= h/T < 2.75:
        alpha_xx = 1.2205 * F_n**6 - 5.4999 * F_n**5 + 5.7966 * F_n**4 + 6.6491 * F_n**3 - 16.123 * F_n**2 + 9.2016 * F_n - 0.6342
    elif 2.75 <= h/T < 3.25:
        alpha_xx = -0.4085 * F_n**6 + 4.534 * F_n**5 - 18.443 * F_n**4 + 35.744 * F_n**3 - 34.381 * F_n**2 + 15.042 * F_n - 1.3807
    elif 3.25 <= h/T < 3.75:
        alpha_xx = 0.4078 * F_n **6 - 0.919 * F_n**5 - 3.8292 * F_n**4 + 15.738 * F_n**3 - 19.766 * F_n**2 + 9.7466 * F_n - 0.6409
    elif 3.75 <= h/T < 4.5:
        alpha_xx = 0.3067 * F_n**6 - 0.3404 * F_n**5 - 5.0511 * F_n**4 + 16.892 * F_n**3 - 20.265 * F_n**2 + 9.9002 * F_n - 0.6712
    elif 4.5 <= h/T < 5.5:
        alpha_xx = 0.3212 * F_n**6 - 0.3559 * F_n**5 - 5.1056 * F_n**4 + 16.926 * F_n**3 - 20.253 * F_n**2 + 10.013 * F_n - 0.7196
    elif 5.5 <= h/T < 6.5:
        alpha_xx = 0.9252 * F_n**6 - 4.2574 * F_n**5 + 5.0363 * F_n **4 + 3.3282 * F_n**3 - 10.367 * F_n**2 + 6.3993 * F_n - 0.2074
    elif 6.5 <= h/T < 7.5:
        alpha_xx = 0.8442 * F_n**6 - 4.0261 * F_n**5 + 5.313 * F_n **4 + 1.6442 * F_n**3 - 8.1848 * F_n**2 + 5.3209 * F_n - 0.0267
    elif 7.5 <= h/T < 8.5:
        alpha_xx = 0.1211 * F_n**6 + 0.628 * F_n**5 - 6.5106 * F_n**4 + 16.7 * F_n**3 - 18.267 * F_n**2 + 8.7077 * F_n - 0.4745
        
                
    elif 8.5 <= h/T < 9.5:
        if F_n < 0.6:
            alpha_xx = 1
        elif F_n >= 0.6:
            alpha_xx = -6.4069 * F_n**6 + 47.308 * F_n**5 - 141.93 * F_n**4 + 220.23 * F_n**3 - 185.05 * F_n**2 + 79.25 * F_n - 12.484
    elif h/T >= 9.5: 
        if F_n < 0.6:
            alpha_xx = 1
        elif F_n >= 0.6:
            alpha_xx = -6.0727 * F_n**6 + 44.97 * F_n**5 - 135.21 * F_n**4 + 210.13 * F_n**3 - 176.72 * F_n**2 + 75.728 * F_n - 11.893
V_2 = V_0 / alpha_xx 
print(F_n,alpha_xx,V_2)


0.5213745913371833 0.9956927310703285 4.017303607007521


In [8]:
#def wave_resistance(V_2):

# This method is based on literature of Sarris (Naval Ship Propulsion and Electric Power Systems Selection for
# Optimal Fuel Consumption) (2003) applying Holtrop and Mennen (pag 26)

F_nL = V_2 / np.sqrt(g*L) #Froude number    


    #parameter c_7 is determined by the B/L ratio
if B/L < 0.11:
    c_7 = 0.229577 * (B/L)**0.33333
elif B/L > 0.25:
    c_7 = 0.5 - 0.0625 * (L/B)
else: 
    c_7 = B/L

#half angle of entrance in degrees
i_E = 1 + 89 * np.exp(-((L/B)**0.80856) * ((1 - C_wp)**0.30484) * ((1 - C_p - 0.0225 * lcb)**0.6367) * ((L_R / B)**0.34574) * ((100 * delta / (L**3))**0.16302))

c_1 = 2223105 * (c_7**3.78613) * ((T/B)**1.07961) * (90 - i_E)**(-1.37165) 
c_2 = 1 #accounts for the effect of the bulbous bow, which is not present at inland ships
c_5 = 1 - (0.8 * A_T) / (B * T * C_M) #influence of the transom stern on the wave resistance

#parameter c_15 depoends on the ratio L^3 / delta
if (L**3) / delta < 512:
    c_15 = -1.69385
elif (L**3) / delta > 1727:
    c_15 = 0
else:
    c_15 = -1.69385 + (L / (delta**(1/3)) - 8) / 2.36

#parameter c_16 depends on C_p
if C_p < 0.8:
    c_16 = 8.07981 * C_p - 13.8673 * (C_p**2) + 6.984388 * (C_p**3)
else:
    c_16 = 1.73014 - 0.7067 * C_p

m_1 = 0.0140407 * (L / T) - 1.75254 * ((delta)**(1/3) / L) - 4.79323 * (B / L) - c_16

m_4 = 0.4 * c_15 * np.exp(-0.034 * (F_nL**(-3.29)))    

if L/B < 12:
    lmbda = 1.446 * C_p - 0.03 * (L/B)
else: 
    lmbda = 1.446 * C_p - 0.36

#parameters needed for RW_2
c_17 = 6919.3 * (C_M**(-1.3346)) * ((delta/ (L**3))**2.00977) * ((L/B - 2)**1.40692)
m_3 = -7.2035 * ((B / L)**0.326869) * ((T/B)**0.605375)


######### When Fn < 0.4
RW_1 = c_1 * c_2 * c_5 * delta * rho * g * np.exp(m_1 * (F_nL**(-0.9)) + m_4 * np.cos(lmbda * (F_nL**(-2)))) 

######## When Fn > 0.5
RW_2 = c_17 * c_2 * c_5 * delta * rho * g * np.exp(m_3 * (F_nL**(-0.9)) + m_4 * np.cos(lmbda * (F_nL**(-2))))

if F_nL < 0.4:
    R_W = RW_1 / 1000 #kN
elif F_nL > 0.55:
    R_W = RW_2 / 1000 #kN
else:
    R_W = (RW_1 + ((10 * F_nL - 4) * (RW_2 - RW_1)) / 1.5) / 1000 #kN
# test Rw and T 
m2=c_15 * (C_p**2) *np.exp((-0.1)* (F_nL**(-2)))                                    
m1x=np.exp(m_1 * (F_nL**(-0.9)) + m_4 * np.cos(lmbda * (F_nL**(-2)))) 
m2x=np.exp(m_1 * (F_nL**(-0.9)) + m2 * np.cos(lmbda * (F_nL**(-2))))
RW_1_HM = c_1 * c_2 * c_5 * delta * rho * g * np.exp(m_1 * (F_nL**(-0.9)) + m2 * np.cos(lmbda * (F_nL**(-2))))/1000 #Holtrop&Mennen's Rw
print(m2,m2x,RW_1_HM)
print(m_1,m_4,m1x)        
print(F_nL,c_1,c_2,c_5,R_W) 

-0.0009131780096481333 0.009489242388660694 40.738819645085385
-0.7026589346384469 -5.687453321542371e-16 0.009497459511266728
0.12229362217970444 0.34939397674338646 1 0.8393594463125402 40.77409705313735


In [9]:
# T=[0.7,0.9,1,1.1,1.2,1.3,1.4,1.6,2,2.2,2.4,2.6,3,3.2,3.5]
# for i in range (len(T)):
#     RW_1=RW_1[i]
#     plt.plot(T[i],RW_1[i])

## (3) Residual resistance terms: Rres

In [10]:
F_nt = V_2 / np.sqrt(2 * g * A_T / (B + B * C_wp))

c_6 = 0.2 * (1 - 0.2 * F_nt)

R_TR = (0.5 * rho * (V_2**2) * A_T * c_6) / 1000 #kN



#Model-ship correlation resistance(R_A)
if T / L < 0.04:
    c_4 = T / L
else:
    c_4 = 0.04
c_2 = 1

C_A = 0.006 * (L + 100)**(-0.16) - 0.00205 + 0.003 * np.sqrt(L / 7.5) * (C_B**4) * c_2 * (0.04 - c_4)


####### Holtrop and Mennen in the document of Sarris, 2003 #######
R_A = (0.5 * rho * (V_2**2) * S_T * C_A) / 1000 #kW
print(R_TR,R_A) 


2.717344459618239 7.085826832767788


## (4) Total resistance:  $R_{T}$

In [11]:
#Total resistance kN

R_tot = Rf_onek1 + R_APP + R_W + R_TR+R_A
print(R_tot)

74.8576912429418


#  Power estimation

## (1) Total required power for propulsion and hotel

In [12]:

# 1 Required power for systems on board
P_hotel = 0.05 * P_installed #5% just an assumption for now


# 2 Required power for propulsion:(V_0 * R_tot1) / (eta_0 * eta_r * eta_h * eta_t * eta_g)

#Calculation hull efficiency:eta_h, based on Van Terwisga (1989)
dw = 0 
    
w = 0.11 * (0.16 / x) * C_B * np.sqrt((delta**(1/3)) / D_s) - dw

if x == 1:
    t = 0.6 * w * (1 + 0.67 * w)
else:
    t = 0.8 * w * (1 + 0.25 * w)
    
eta_h = (1 - t) / (1 - w)


#  Total required power kw

P_tot = P_hotel + (V_0 * R_tot) / (eta_0 * eta_r * eta_h * eta_t * eta_g)
print(P_tot)

626.0286874391381


#  Emission estimation

## (1) General emission factors (engine age)

In [13]:
if c_year < 1974:
    EM_CO2 = 756
    EM_PM10 = 0.6
    EM_NOX = 10.8
elif 1975 <= c_year <= 1979:
    EM_CO2 = 730
    EM_PM10 = 0.6
    EM_NOX = 10.6
elif 1980 <= c_year <= 1984:
    EM_CO2 = 714
    EM_PM10 = 0.6
    EM_NOX = 10.4
elif 1985 <= c_year <= 1989:
    EM_CO2 = 698
    EM_PM10 = 0.5
    EM_NOX = 10.1
elif 1990 <= c_year <= 1994:
    EM_CO2 = 698
    EM_PM10 = 0.4
    EM_NOX = 10.1
elif 1995 <= c_year <= 2002:
    EM_CO2 = 650
    EM_PM10 = 0.3
    EM_NOX = 9.4
elif 2003 <= c_year <= 2007:
    EM_CO2 = 635
    EM_PM10 = 0.3
    EM_NOX = 9.2
elif 2008 <= c_year <= 2019:
    EM_CO2 = 635
    EM_PM10 = 0.2
    EM_NOX = 7
elif c_year > 2019:
    if L == 1:
        EM_CO2 = 650
        EM_PM10 = 0.1
        EM_NOX = 2.9
    else:
        EM_CO2 = 603
        EM_PM10 = 0.015
        EM_NOX = 2.4
P_partial = P_tot / P_installed
print(EM_CO2, EM_PM10, EM_NOX,P_partial) 


603 0.015 2.4 0.35773067853665036


## (2) correction emission factors (partial engine load)

In [14]:
corf = pd.read_excel (r'correctionfactors1.xls')

corf_CO2 = 0
corf_PM10 = 0
corf_NOX = 0

#If P_partial is <= 5%

if P_partial <= 0.05:
    corf_CO2 = corf.iloc[0, 5]
    corf_PM10 = corf.iloc[0, 6]

    if c_year < 2008:
        corf_NOX = corf.iloc[0, 1] #<= CCR-1 class
    elif 2008 <= c_year <= 2019:
        corf_NOX = corf.iloc[0, 2] #CCR-2 / Stage IIIa
    elif c_year > 2019:
            if L_w == 1:
                corf_NOX = corf.iloc[0, 3] #Stage V: IWP/IWA-v/c-3 class (vessels with P <300 kW: assumed to be weight class L1)
            else:
                corf_NOX = corf.iloc[0, 4]  #Stage V:IWP/IWA-v/c-4 class (vessels with P >300 kw: assumed to be weight class L2-L3)

#If P_parital = 100% 

elif P_partial == 1:
    corf_CO2 = corf.iloc[19, 5]
    corf_PM10 = corf.iloc[19, 6]

    if c_year < 2008:
        corf_NOX = corf.iloc[19, 1] #<= CCR-1 class
    elif 2008 <= c_year <= 2019:
        corf_NOX = corf.iloc[19, 2] #CCR-2 / Stage IIIa
    elif c_year > 2019:
        if L_w == 1:
            corf_NOX = corf.iloc[19, 3] #Stage V: IWP/IWA-v/c-3 class (vessels with P <300 kW: assumed to be weight class L1)
        else:
            corf_NOX = corf.iloc[19, 4]  #Stage V:IWP/IWA-v/c-4 class (vessels with P >300 kw: assumed to be weight class L2-L3) 

#If P_partial is inbetween 5% and 100%. 

elif 0.05 < P_partial < 1:

    #It is determined inbetween which two percentages in the table the partial engine load lies
    #The correction factor is determined by means of linear interpolation  

    for i in range(len(corf)):
        if corf.iloc[i, 0] < P_partial <= corf.iloc[i + 1, 0]: 

            corf_CO2 = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 5] - corf.iloc[i, 5])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 5]
            corf_PM10 = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 6] - corf.iloc[i, 6])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 6]

            if c_year < 2008:
                corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 1] - corf.iloc[i, 1])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 1]
            elif 2008 <= c_year <= 2019:
                corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 2] - corf.iloc[i, 2])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 2]
            elif c_year > 2019:
                if L_w == 1:
                    corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 3] - corf.iloc[i, 3])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 3]
                else:
                    corf_NOX = ((P_partial - corf.iloc[i, 0]) * (corf.iloc[i+1, 4] - corf.iloc[i, 4])) / (corf.iloc[i+1, 0] - corf.iloc[i, 0]) + corf.iloc[i, 4] 
print(corf_CO2, corf_PM10, corf_NOX) 

1.08690772858534 1.0469077285853399 1.6191770500486893


## (3)  emission rate(g/km) 

In [15]:
#emission rate [g/km]
emission_g_m_CO2 = P_tot * EM_CO2 * corf_CO2/  V_0_km
emission_g_m_PM10 = P_tot * EM_PM10 * corf_PM10/  V_0_km
emission_g_m_NOX = P_tot * EM_NOX * corf_NOX/  V_0_km
print(emission_g_m_CO2,
      emission_g_m_PM10,
      emission_g_m_NOX)

28493.23315780017 0.6827023658293434 168.94188056225943


## (4) total emission (kg)

In [16]:
#total emissions in the waterway section  kg

total_emission_g_m_CO2 = emission_g_m_CO2*distance/1000
total_emission_g_m_PM10 = emission_g_m_PM10*distance/1000
total_emission_g_m_NOX =emission_g_m_NOX*distance/1000
print(total_emission_g_m_CO2,total_emission_g_m_PM10,total_emission_g_m_NOX)


5100.28873524623 0.12220372348345246 30.24059662064444


## required energy （kwh）

In [17]:
# required energy in the waterway section（kwh）
Energy=P_tot*(distance/V_0_km)
print(Energy)

7781.884378583731


##  fuel consumption (kg diesel) 

In [18]:
# fuel consumption in the waterway section (kg diesel)
Fuel=Energy*0.225
print(Fuel)

1750.9239851813395
